In [106]:
import pandas as pd
import numpy as np
import ccxt
from datetime import datetime


ex = ccxt.binance()
timeframe = '1m'
since = ex.parse8601('2023-01-01T00:00:00Z')
limit = 0 # how many data point you need (0 => fetch all ohlcv from since towards "NOW")

def fetch_history_ohlcvs(symbol, timeframe, since, limit=0):
    all_ohlcvs = []
    local_since = since
    while True:
        try:
            ohlcvs = ex.fetch_ohlcv(symbol, timeframe, local_since)
            all_ohlcvs += ohlcvs
            if len(ohlcvs):
                #print('Fetched', len(ohlcvs), symbol, timeframe, 'candles from', bi.iso8601(ohlcvs[0][0]))
                local_since = ohlcvs[-1][0] + 1
            else:
                break
            
            if limit != 0 and len(all_ohlcvs) > limit:
                break
        except Exception as e:
            print(type(e).__name__, str(e))
    print('Fetched', len(all_ohlcvs), symbol, timeframe, 'candles in total')
    return all_ohlcvs

In [107]:
base = pd.DataFrame(fetch_history_ohlcvs('BNB/USDT', timeframe, since, limit), columns=list('TOHLCV'))

Fetched 86674 BNB/USDT 1m candles in total


In [108]:
coin = pd.DataFrame(fetch_history_ohlcvs('ENS/USDT', timeframe, since, limit), columns=list('TOHLCV'))

Fetched 86676 ENS/USDT 1m candles in total


In [153]:
merged_price = base.merge(coin, on='T')[['C_x', 'C_y', 'T']]
merged_price.index = merged_price['T']
merged_price

,C_x,C_y,T
T,,,
1672531200000,246.3,10.79,1672531200000
1672531260000,246.3,10.79,1672531260000
1672531320000,246.2,10.78,1672531320000
1672531380000,246.0,10.76,1672531380000
1672531440000,246.2,10.78,1672531440000
...,...,...,...
1677731340000,300.1,15.73,1677731340000
1677731400000,300.2,15.74,1677731400000
1677731460000,300.2,15.74,1677731460000


In [154]:
Length = 240
merged_price["std_x"] = merged_price.C_x.rolling(Length).std()
merged_price["mean_x"] = merged_price.C_x.rolling(Length).mean()
merged_price["zscore_x"] = (merged_price.C_x - merged_price.mean_x)/merged_price.std_x

merged_price["std_y"] = merged_price.C_y.rolling(Length).std()
merged_price["mean_y"] = merged_price.C_y.rolling(Length).mean()
merged_price["zscore_y"] = (merged_price.C_y - merged_price.mean_y)/merged_price.std_y

merged_price["zscore_diff"] = merged_price.zscore_x - merged_price.zscore_y
merged_price.tail(10)

,C_x,C_y,T,std_x,mean_x,zscore_x,std_y,mean_y,zscore_y,zscore_diff
T,,,,,,,,,,
1677731040000,300.1,15.71,1677731040000,1.350803,300.770833,-0.496618,0.135481,15.809042,-0.731036,0.234418
1677731100000,300.1,15.71,1677731100000,1.341495,300.757500,-0.490125,0.133594,15.807125,-0.727016,0.236892
1677731160000,300.1,15.72,1677731160000,1.332775,300.744583,-0.483640,0.131505,15.805208,-0.647945,0.164305
1677731220000,300.1,15.72,1677731220000,1.323074,300.731250,-0.477109,0.129475,15.803333,-0.643626,0.166517
1677731280000,300.1,15.74,1677731280000,1.313166,300.717917,-0.470555,0.127088,15.801458,-0.483588,0.013033
1677731340000,300.1,15.73,1677731340000,1.303046,300.704583,-0.463977,0.125261,15.799750,-0.556837,0.092860
1677731400000,300.2,15.74,1677731400000,1.292533,300.691667,-0.380390,0.123362,15.798083,-0.470838,0.090448
1677731460000,300.2,15.74,1677731460000,1.282642,300.679167,-0.373578,0.121526,15.796458,-0.464578,0.091000
1677731520000,300.3,15.74,1677731520000,1.269764,300.665833,-0.288111,0.119009,15.794625,-0.459001,0.170889


In [155]:
merged_price.drop(columns = ['std_x', 'std_y', 'mean_x', 'mean_y', 'zscore_x', 'zscore_y', 'T'], inplace = True)
merged_price.dropna(inplace = True)

In [156]:
merged_price

,C_x,C_y,zscore_diff
T,,,
1672545540000,245.3,10.69,0.458105
1672545600000,245.3,10.71,-0.266257
1672545660000,245.2,10.71,-0.543921
1672545720000,245.3,10.71,-0.259412
1672545780000,245.3,10.72,-0.626114
...,...,...,...
1677731340000,300.1,15.73,0.092860
1677731400000,300.2,15.74,0.090448
1677731460000,300.2,15.74,0.091000


In [162]:
side = 'flat' ## long, short, flat
long_price = 1
short_price = 1
Fence = 5

profit_count = 0
loss_count = 0
profit_amount = 0
loss_amount = 0

for index, row in merged_price.iterrows():
#     print(row.C_x, row.C_y, row.zscore_diff)
    DateTime = datetime.fromtimestamp(index/1000)
    
    if row.zscore_diff >= Fence and side == 'flat': # short base, long coin
        short_price = row.C_x
        long_price = row.C_y
        side = 'long'
        print(f"{DateTime.year}/{DateTime.month}/{DateTime.day} {DateTime.hour}:{DateTime.minute}", end=" || ")
        print(f"Short Base at {short_price} || Long Coin at {long_price}")
    elif row.zscore_diff <= -Fence and side == 'flat': # short coin, long base
        short_price = row.C_y
        long_price = row.C_x
        side = 'short'
        print(f"{DateTime.year}/{DateTime.month}/{DateTime.day} {DateTime.hour}:{DateTime.minute}", end=" || ")
        print(f"Long Base at {long_price}  || Short Coin at {short_price}")
    
    elif side == 'long' and (short_price-row.C_x)/short_price + (row.C_y-long_price)/long_price >= 3/100: ## Long Profit(Close)
        print(f"{DateTime.year}/{DateTime.month}/{DateTime.day} {DateTime.hour}:{DateTime.minute}", end=" || ")
        print(f"Close Short at {row.C_x}  || Close Long at {row.C_y} || Profit {(short_price-row.C_x)/short_price + (row.C_y-long_price)/long_price}")
        profit_amount += (short_price-row.C_x)/short_price + (row.C_y-long_price)/long_price
        long_price = 1
        short_price = 1
        side = 'flat'
        profit_count += 1
    
    elif side == 'short' and (row.C_x-long_price)/long_price + (short_price-row.C_y)/short_price >= 3/100:  ## Short Profit(Close)
        print(f"{DateTime.year}/{DateTime.month}/{DateTime.day} {DateTime.hour}:{DateTime.minute}", end=" || ")
        print(f"Close Long at {row.C_x}  || Close Short at {row.C_y} || Profit {(row.C_x-long_price)/long_price + (short_price-row.C_y)/short_price}")
        profit_amount += (row.C_x-long_price)/long_price + (short_price-row.C_y)/short_price
        long_price = 1
        short_price = 1
        side = 'flat'
        profit_count += 1
    
    elif side == 'long' and (short_price-row.C_x)/short_price + (row.C_y-long_price)/long_price <= -8/100: ## Long Loss (Close)
        print(f"{DateTime.year}/{DateTime.month}/{DateTime.day} {DateTime.hour}:{DateTime.minute}", end=" || ")
        print(f"Close Short at {row.C_x}  || Close Long at {row.C_y} || Loss {(short_price-row.C_x)/short_price + (row.C_y-long_price)/long_price}")
        loss_amount += (short_price-row.C_x)/short_price + (row.C_y-long_price)/long_price
        long_price = 1
        short_price = 1
        side = 'flat'
        loss_count += 1
    
    elif side == 'short' and (row.C_x-long_price)/long_price + (short_price-row.C_y)/short_price <= -8/100: ## Short Loss (Close)
        print(f"{DateTime.year}/{DateTime.month}/{DateTime.day} {DateTime.hour}:{DateTime.minute}", end=" || ")
        print(f"Close Long at {row.C_x}  || Close Short at {row.C_y} || LOSS {(row.C_x-long_price)/long_price + (short_price-row.C_y)/short_price}")
        loss_amount += (row.C_x-long_price)/long_price + (short_price-row.C_y)/short_price
        long_price = 1
        short_price = 1
        side = 'flat'
        loss_count += 1
        
print(f"\n\nTotal: {profit_count + loss_count}\nWin Rate: {profit_count/(profit_count+loss_count)*100}%\nProfit: {profit_amount*100}%\nLoss: {loss_amount*100}%")

2023/1/2 10:43 || Long Base at 244.0  || Short Coin at 10.88
2023/1/6 13:22 || Close Long at 257.9  || Close Short at 11.17 || Profit 0.030312801350048206
2023/1/7 11:56 || Short Base at 262.6 || Long Coin at 11.56
2023/1/10 3:58 || Close Short at 281.9  || Close Long at 12.78 || Profit 0.032040521060357374
2023/1/13 19:52 || Long Base at 287.3  || Short Coin at 13.0
2023/1/14 20:21 || Close Long at 300.1  || Close Short at 13.14 || Profit 0.033783501566306995
2023/1/29 11:16 || Short Base at 305.2 || Long Coin at 15.66
2023/1/31 10:42 || Close Short at 306.6  || Close Long at 16.25 || Profit 0.033088450677821006
2023/2/1 4:14 || Long Base at 311.9  || Short Coin at 16.45
2023/2/1 7:5 || Close Long at 313.5  || Close Short at 16.03 || Profit 0.03066176420429348
2023/2/1 15:36 || Long Base at 311.1  || Short Coin at 16.26
2023/2/1 20:9 || Close Long at 306.2  || Close Short at 15.5 || Profit 0.030989904884584023
2023/2/3 0:19 || Long Base at 329.1  || Short Coin at 16.95
2023/2/3 11:27 